In [6]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import warnings
import seaborn as sns
from sklearn.linear_model import LinearRegression

### Time related preprocessing
- fetch from BigQuery
    - DATETIME_TRUNC: cut data
    - EXTRACT => MONTH, WEEK, DAY, HOUR can be extracted
    - FORMAT_DATETIME => which day can be extracted (1 = Monday, 0 = Monday in Pandas)

In [15]:
# import GCP Client library
from google.cloud import bigquery

#instantiate a client for bigquery service
bigquery_client = bigquery.Client()
bigquery_client

In [19]:
query = """
SELECT
    *,
    EXTRACT(MONTH FROM pickup_hour) AS month,
    EXTRACT(DAY FROM pickup_hour) AS day,
    CAST(format_datetime('%u', pickup_hour) AS INT64) -1 AS weekday,
    EXTRACT(HOUR FROM pickup_hour) AS hour,
    CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_hour) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend
FROM (
    SELECT 
        DATETIME_TRUNC(pickup_datetime, hour) AS pickup_hour,
        count(*) AS cnt
    FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015` 
    WHERE EXTRACT(MONTH from pickup_datetime) = 1
    GROUP BY pickup_hour
)
ORDER BY pickup_hour
"""

In [20]:
# run query and get data from bigquery
query_job = bigquery_client.query(query) #API request

In [21]:
# data into dataframe
df = query_job.to_dataframe()
df.head()

,pickup_hour,cnt,month,day,weekday,hour,is_weekend
0,2015-01-01 00:00:00,28312,1,1,3,0,0
1,2015-01-01 01:00:00,31707,1,1,3,1,0
2,2015-01-01 02:00:00,28068,1,1,3,2,0
3,2015-01-01 03:00:00,24288,1,1,3,3,0
4,2015-01-01 04:00:00,17081,1,1,3,4,0


In [22]:
df.tail()

,pickup_hour,cnt,month,day,weekday,hour,is_weekend
739,2015-01-31 19:00:00,32436,1,31,5,19,1
740,2015-01-31 20:00:00,27555,1,31,5,20,1
741,2015-01-31 21:00:00,27477,1,31,5,21,1
742,2015-01-31 22:00:00,29862,1,31,5,22,1
743,2015-01-31 23:00:00,29856,1,31,5,23,1
